## 김포 제주 노선 가격 정보 읽어오기
- 대상 사이트 : 티웨이 웹사이트

### 테웨이 사이트 웹 크롤링
    - 티웨이 항공권예매 페이지를 이용한 크롤링
    - 세션을 생성하여 세션을 이용해 HTML 페이지 정보 획득
    - 필요 데이터 추출 하여 Pandas 의 DataFrame 형태로 생성
    - 생성된 데이터 엑셀 파일로 저장

In [129]:
from IPython.display import display
import requests
import pandas as pd
import numpy as np
from pandas import DataFrame
from bs4 import BeautifulSoup
import time
from datetime import datetime
from datetime import timedelta
from common.crawling_util import session_crawling

dom_int='D' ## 국내선(D) 국제선(I) 구분
def crawling_TW_data(dpt,arr,dpt_date):
    ##출발지, 도착지, 출발일을 기준으로 국내선(국제선) 편도 가격 읽어오기
    print('Crawling twayair homepage schedule site')
    session_url = "https://www.twayair.com/booking/availabilityList.do"
    session_head = {
        'Referer':'https://www.twayair.com/main.do',
    }
    
    url = 'https://www.twayair.com/booking/ajax/searchAvailability.do'
    head = {
        'Referer':'https://www.twayair.com/booking/availabilityList.do',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'
    }
    param ={
        'origin':dpt,                  'destination':arr,
        'origin1':dpt,                 'destination1':arr,
        'origin2':arr,                 'destination2':dpt,
        'onwardDateStr':dpt_date,      'returnDateStr':dpt_date,
        'today':datetime.today().strftime('%Y%m%d'),
        #'searchAvailId':searchAvailId,
        'travelType':'OW',#'RT',
        'currencyCode':'KRW',            'domesticYn':'Y',
        'pointOfPurchase':'KR',          'paxTypeCountStr':'1,0,0',
        'searchType':'byDate',
        'orderByOW':'',               'orderByRT':'',
        'fareBasisCodeOW':'',         'fareBasisCodeRT':'',
        'arrivCntryCode':'',          'promotionCode':'',
    }

    return session_crawling(session_url,url,param,session_head=session_head,head=head,method='get',json=False)

data_heads = ['Flt','start','end','fare1','fare2','tax1','tax2','seat']
## raw 데이터로 부터 DataFrame 생성
def raw_to_df(raw_data):
    soup = BeautifulSoup(raw_data,'lxml')
    fare_data = []
    for tr in soup.select('#tbodyOnward tr'):
        td = tr.select('td')
        td_list = [f.text.strip() for f in td[:3]]
        ## 스마트운임에서 항공료만 가져온다.
        fares = [f.attrs['value'] for f in td[4].select('input') if f.attrs['type'] == 'hidden' and f.attrs['name'] == 'fare']
        ## 좌석 체크 하여 매진인 경우 가격 0 처리
        soldout = td[4].select('.txt3')[0].select('.soldout')
        if len(soldout) == 0: #좌석이 많음
            td_list.extend(fares)
        else:
            soldout_txt = soldout[0].text.strip()
            if '매진' in soldout_txt: ## 매진
                td_list.append('0')
            else:
                td_list.extend(fares)
        ## 일반운잉에서 항공료, 유류세, 공항세를 가져온다.
        fares = [f.attrs['value'] for f in td[5].select('input') 
                 if f.attrs['type'] == 'hidden' and f.attrs['name'] in ['fare','surcharge','tax']]
        ## 좌석 체크 하여 매진인 경우 가격 0 처리
        soldout = td[5].select('.txt3')[0].select('.soldout')
        if len(soldout) == 0: #좌석이 많음
            td_list.extend(fares)
            td_list.append('9')
        else:
            soldout_txt = soldout[0].text.strip()
            if '매진' in soldout_txt: #매진
                td_list.extend(['0','0','0','0'])
            else:
                td_list.extend(fares)
                td_list.append(soldout_txt.replace('(','').replace('석','').replace(')',''))
        fare_data.append(td_list)
    return DataFrame(fare_data,columns=data_heads)

## 하루 기준 가격정보, 텍스정보, 최소값, 최대값, 평균 DataFrame 생성
def read_TW_1day_fare(dpt,arr,dpt_date):
    ## 데이터 읽어오기
    raw_data = crawling_TW_data(dpt,arr,dpt_date)
    df=raw_to_df(raw_data)
    if df is None or len(df) == 0: ## 읽는 도중 에러가 나거나 익셉션 발생으로 문제가 있을 경우 처리
        print('********** No Data Type 1 **********')
        return None

    ## 최소값 최대값 평균 계산
    fare_arr = df[['fare1','fare2']].values ## fare 만 구해오기
    fare_arr = fare_arr.flatten() # shape 1차원으로 변경
    fare_arr = np.unique(fare_arr)
    if '' in fare_arr or '0' in fare_arr:
        fare_arr = fare_arr[1:] # 0 값 제거
    fare_arr = fare_arr.astype('float') # 중복값 제거, 수치형으로 변경
    ## 요약정보 추가
    df.ix[len(df)] = [dpt_date,'min','max','mean',str(fare_arr.min()),str(fare_arr.max()),str(fare_arr.mean()),'']
    return df

## 정해진 기간의 데이터 읽어오기, 기본 30일
def read_TW_date_range_fare(dpt,arr,start=0,end=31):
    # 기본 30일간 데이터 읽어서 파일로 저장
    date_range = [ (datetime.today()+timedelta(1)*i).strftime('%Y%m%d') for i in range(start,end)]
    df_list = []
    for d in date_range:
        try:
            fare_df = read_TW_1day_fare(dpt,arr,d)
            if fare_df is not None:
                df_list.append(fare_df)
        except Exception as e:
            print('****** Error occured : ',e)
    result = pd.concat(df_list,ignore_index=True)
    print('++++++++++Total : ', len(result))
    ## 파일 저장
    result.to_excel('{}/{}_{}_{}_{}_{}_{}.xls'.format('excel','TW',dpt,arr,start,end,datetime.today().strftime('%Y%m%d%H%m')))
    return result

In [127]:
## 하루치 데이터 읽어오기
dpt, arr, dpt_date = 'GMP','CJU','20170520'
#dpt, arr, dpt_date, dom_int = 'ICN','NRT','2017-05-01','I'

read_TW_1day_fare(dpt,arr,dpt_date)

Crawling twayair homepage schedule site
Start Session crawling
make session :  https://www.twayair.com/booking/availabilityList.do
crawling :  https://www.twayair.com/booking/ajax/searchAvailability.do
>> Parameters
searchType:byDate , origin:GMP , fareBasisCodeRT: , today:20170424 , currencyCode:KRW , arrivCntryCode: , fareBasisCodeOW: , domesticYn:Y , destination1:CJU , returnDateStr:20170520 , pointOfPurchase:KR , orderByOW: , travelType:OW , destination:CJU , orderByRT: , paxTypeCountStr:1,0,0 , origin2:CJU , promotionCode: , destination2:GMP , onwardDateStr:20170520 , origin1:GMP , 
End Session crawling


,Flt,start,end,fare1,fare2,tax1,tax2,seat
0,TW741,06:40,07:50,93900.0,97700.0,2200.0,4000.0,9
1,TW701,07:00,08:10,0,97700.0,2200.0,4000.0,9
2,TW703,08:15,09:25,0,97700.0,2200.0,4000.0,3
3,TW705,09:45,10:55,0,0,0,0,0
4,TW707,10:30,11:40,0,97700.0,2200.0,4000.0,9
5,TW709,11:45,12:55,93900.0,97700.0,2200.0,4000.0,9
6,TW711,12:50,14:00,0,80000.0,2200.0,4000.0,9
7,TW717,13:20,14:30,76900.0,80000.0,2200.0,4000.0,9
8,TW745,13:25,14:35,0,80000.0,2200.0,4000.0,9
9,TW713,14:05,15:15,72900.0,80000.0,2200.0,4000.0,9


In [132]:
## 정해진 기간의 데이터 읽어오기
dpt, arr, = 'GMP','CJU'
start,end = 0, 31 ## 읽어올 데이터 기간
read_TW_date_range_fare(dpt,arr,start,end)

Crawling twayair homepage schedule site
Start Session crawling
make session :  https://www.twayair.com/booking/availabilityList.do
crawling :  https://www.twayair.com/booking/ajax/searchAvailability.do
>> Parameters
searchType:byDate , origin:GMP , fareBasisCodeRT: , today:20170424 , currencyCode:KRW , arrivCntryCode: , fareBasisCodeOW: , domesticYn:Y , destination1:CJU , returnDateStr:20170424 , pointOfPurchase:KR , orderByOW: , travelType:OW , destination:CJU , orderByRT: , paxTypeCountStr:1,0,0 , origin2:CJU , promotionCode: , destination2:GMP , onwardDateStr:20170424 , origin1:GMP , 
End Session crawling
Crawling twayair homepage schedule site
Start Session crawling
make session :  https://www.twayair.com/booking/availabilityList.do
crawling :  https://www.twayair.com/booking/ajax/searchAvailability.do
>> Parameters
searchType:byDate , origin:GMP , fareBasisCodeRT: , today:20170424 , currencyCode:KRW , arrivCntryCode: , fareBasisCodeOW: , domesticYn:Y , destination1:CJU , returnDat

,Flt,start,end,fare1,fare2,tax1,tax2,seat
0,TW711,12:50,14:00,0,0,0,0,0
1,TW717,13:20,14:30,0,0,0,0,0
2,TW745,14:35,15:45,46900.0,65600.0,2200.0,4000.0,2
3,TW715,15:15,16:25,36900.0,65600.0,2200.0,4000.0,9
4,TW723,17:25,18:35,29900.0,65600.0,2200.0,4000.0,9
5,TW747,18:10,19:20,23900.0,65600.0,2200.0,4000.0,9
6,TW771,18:25,19:35,23900.0,65600.0,2200.0,4000.0,9
7,TW721,18:45,19:55,23900.0,65600.0,2200.0,4000.0,9
8,20170424,min,max,mean,23900.0,65600.0,40640.0,
9,TW741,06:40,07:50,43900.0,65600.0,2200.0,4000.0,9


In [133]:
## 정해진 기간의 데이터 읽어오기
dpt, arr, = 'GMP','CJU'
start,end = 31, 46 ## 읽어올 데이터 기간
read_TW_date_range_fare(dpt,arr,start,end)

Crawling twayair homepage schedule site
Start Session crawling
make session :  https://www.twayair.com/booking/availabilityList.do
crawling :  https://www.twayair.com/booking/ajax/searchAvailability.do
>> Parameters
searchType:byDate , origin:GMP , fareBasisCodeRT: , today:20170424 , currencyCode:KRW , arrivCntryCode: , fareBasisCodeOW: , domesticYn:Y , destination1:CJU , returnDateStr:20170525 , pointOfPurchase:KR , orderByOW: , travelType:OW , destination:CJU , orderByRT: , paxTypeCountStr:1,0,0 , origin2:CJU , promotionCode: , destination2:GMP , onwardDateStr:20170525 , origin1:GMP , 
End Session crawling
Crawling twayair homepage schedule site
Start Session crawling
make session :  https://www.twayair.com/booking/availabilityList.do
crawling :  https://www.twayair.com/booking/ajax/searchAvailability.do
>> Parameters
searchType:byDate , origin:GMP , fareBasisCodeRT: , today:20170424 , currencyCode:KRW , arrivCntryCode: , fareBasisCodeOW: , domesticYn:Y , destination1:CJU , returnDat

,Flt,start,end,fare1,fare2,tax1,tax2,seat
0,TW741,06:40,07:50,49900.0,65600.0,2200.0,4000.0,9
1,TW701,07:00,08:10,56900.0,65600.0,2200.0,4000.0,9
2,TW703,08:15,09:25,59900.0,65600.0,2200.0,4000.0,9
3,TW705,09:45,10:55,62900.0,65600.0,2200.0,4000.0,9
4,TW707,10:30,11:40,59900.0,65600.0,2200.0,4000.0,9
5,TW709,11:45,12:55,59900.0,65600.0,2200.0,4000.0,9
6,TW711,12:50,14:00,56900.0,65600.0,2200.0,4000.0,9
7,TW717,13:20,14:30,53900.0,65600.0,2200.0,4000.0,9
8,TW713,14:05,15:15,46900.0,65600.0,2200.0,4000.0,9
9,TW745,14:35,15:45,46900.0,65600.0,2200.0,4000.0,9


In [134]:
## 정해진 기간의 데이터 읽어오기
dpt, arr, = 'GMP','CJU'
start,end = 46, 90 ## 읽어올 데이터 기간
read_TW_date_range_fare(dpt,arr,start,end)

Crawling twayair homepage schedule site
Start Session crawling
make session :  https://www.twayair.com/booking/availabilityList.do
crawling :  https://www.twayair.com/booking/ajax/searchAvailability.do
>> Parameters
searchType:byDate , origin:GMP , fareBasisCodeRT: , today:20170424 , currencyCode:KRW , arrivCntryCode: , fareBasisCodeOW: , domesticYn:Y , destination1:CJU , returnDateStr:20170609 , pointOfPurchase:KR , orderByOW: , travelType:OW , destination:CJU , orderByRT: , paxTypeCountStr:1,0,0 , origin2:CJU , promotionCode: , destination2:GMP , onwardDateStr:20170609 , origin1:GMP , 
End Session crawling
Crawling twayair homepage schedule site
Start Session crawling
make session :  https://www.twayair.com/booking/availabilityList.do
crawling :  https://www.twayair.com/booking/ajax/searchAvailability.do
>> Parameters
searchType:byDate , origin:GMP , fareBasisCodeRT: , today:20170424 , currencyCode:KRW , arrivCntryCode: , fareBasisCodeOW: , domesticYn:Y , destination1:CJU , returnDat

,Flt,start,end,fare1,fare2,tax1,tax2,seat
0,TW701,07:00,08:10,93900.0,97700.0,2200.0,4000.0,9
1,TW703,08:15,09:25,0,97700.0,2200.0,4000.0,9
2,TW705,09:45,10:55,0,97700.0,2200.0,4000.0,9
3,TW709,11:45,12:55,93900.0,97700.0,2200.0,4000.0,9
4,TW711,12:50,14:00,84900.0,97700.0,2200.0,4000.0,9
5,TW717,13:20,14:30,84900.0,97700.0,2200.0,4000.0,9
6,TW745,14:35,15:45,69900.0,97700.0,2200.0,4000.0,9
7,TW715,15:15,16:25,69900.0,97700.0,2200.0,4000.0,9
8,TW723,17:25,18:35,69900.0,97700.0,2200.0,4000.0,9
9,TW747,18:10,19:20,69900.0,97700.0,2200.0,4000.0,9
